In [1]:
import requests 
import datetime
import pandas as pd
import altair as alt
import json


from dotenv import load_dotenv
import os

load_dotenv()

# api_key=os.getenv("WEATHERAPI_APIKEY")

True

In [2]:
# Historical data does not work with the free key
# response = requests.get(f"https://api.weatherapi.com/v1/current.json?key={api_key}&q=Berlin&dt=2015-01-01")
# berlin = response.json()

In [3]:
# I thought about doing an API-Call, but downloading the csv was just so much easier. 

# response = requests.get('https://archive-api.open-meteo.com/v1/archive?latitude=52.52&longitude=13.405&start_date=2015-06-16&end_date=2025-06-15&daily=temperature_2m_mean,rain_sum,snowfall_sum,sunshine_duration')

# berlin = response.json()

# response = requests.get('https://archive-api.open-meteo.com/v1/archive?latitude=40.6782&longitude=73.9442&start_date=2015-06-16&end_date=2025-06-15&daily=temperature_2m_mean,rain_sum,snowfall_sum,sunshine_duration')

# new_york = response.json()

Data source: https://open-meteo.com/

This describtion works for both dataframes - one will have berlin added, the other one will have brooklyn added

day: date of the measures
temperature: mean temperature on that day in Celcius
rain: sum of rain on that day in millimeter
snowfall: sum of snowfall on that day in Centimeter
sunshine: duration of sunshine in that day in hours (converted von seconds to hours)

In [4]:
df = pd.read_csv('raw/weather_berlin.csv', header=2)

In [5]:
# rename columns

(
    df
    .rename(
        columns={
            'time': 'day', 
            'temperature_2m_mean (°C)' : 'temperature', 
            'rain_sum (mm)' : 'rain', 
            'snowfall_sum (cm)': 'snowfall', 
            'sunshine_duration (s)':'sunshine',
            'weather_code (wmo code)' : 'weather_code',
            'relative_humidity_2m_mean (%)' : 'humidity',
            'apparent_temperature_mean (°C)' : 'feels_like',
            'daylight_duration (s)' : 'daylight_duration'
        }, 
        inplace=True
    )
)

In [6]:
df

,day,weather_code,humidity,temperature,feels_like,daylight_duration,sunshine,rain,snowfall
0,2015-07-15,3,71,18.3,17.6,58910.30,41495.63,0.0,0.0
1,2015-07-16,51,67,20.3,20.5,58773.57,46595.22,0.1,0.0
2,2015-07-17,3,67,23.1,24.1,58633.02,50924.75,0.0,0.0
3,2015-07-18,51,67,22.7,23.2,58488.84,52679.86,0.3,0.0
4,2015-07-19,63,80,18.7,18.9,58341.02,26400.53,11.3,0.0
...,...,...,...,...,...,...,...,...,...
3619,2025-06-11,51,67,15.8,13.3,60324.71,48035.18,1.1,0.0
3620,2025-06-12,1,58,16.6,15.2,60383.70,55367.75,0.0,0.0
3621,2025-06-13,3,42,18.7,15.9,60436.27,55407.55,0.0,0.0
3622,2025-06-14,0,42,23.0,21.0,60482.27,55486.68,0.0,0.0


In [7]:
# calculate sunshine from seconds to hours

df['sunshine_hours'] = df['sunshine'] / 3600
df = df.drop('sunshine', axis=1)

In [8]:
# calculate daylight from seconds to hours

df['daylight_hours'] = df['daylight_duration'] / 3600
df = df.drop('daylight_duration', axis=1)

In [9]:
# add month to data
df['day'] = pd.to_datetime(df['day'])

In [10]:
df['month_year'] = df['day'].dt.to_period('M')
df['year'] = df['day'].dt.to_period('Y')

In [11]:
# add season according to month
# Dez, Jan, Feb == Winter
# Mar, Apr, Mai == Spring
# Jun, Jul, Aug == Summer
# Sep, Oct, Nov == Fall

df['month_num'] = df['day'].dt.month

def get_season(month):
    if month in [12, 1, 2]:
        return 'winter'
    elif month in [3, 4, 5]:
        return 'spring'
    elif month in [6, 7, 8]:
        return 'summer'
    elif month in [9, 10, 11]:
        return 'fall'

df['season'] = df['month_num'].apply(get_season)

In [12]:
df.dtypes

day               datetime64[ns]
weather_code               int64
humidity                   int64
temperature              float64
feels_like               float64
rain                     float64
snowfall                 float64
sunshine_hours           float64
daylight_hours           float64
month_year             period[M]
year               period[Y-DEC]
month_num                  int32
season                    object
dtype: object

In [13]:
df['day_of_week'] = df['day'].dt.day_name()

In [14]:
df

,day,weather_code,humidity,temperature,feels_like,rain,snowfall,sunshine_hours,daylight_hours,month_year,year,month_num,season,day_of_week
0,2015-07-15,3,71,18.3,17.6,0.0,0.0,11.526564,16.363972,2015-07,2015,7,summer,Wednesday
1,2015-07-16,51,67,20.3,20.5,0.1,0.0,12.943117,16.325992,2015-07,2015,7,summer,Thursday
2,2015-07-17,3,67,23.1,24.1,0.0,0.0,14.145764,16.286950,2015-07,2015,7,summer,Friday
3,2015-07-18,51,67,22.7,23.2,0.3,0.0,14.633294,16.246900,2015-07,2015,7,summer,Saturday
4,2015-07-19,63,80,18.7,18.9,11.3,0.0,7.333481,16.205839,2015-07,2015,7,summer,Sunday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3619,2025-06-11,51,67,15.8,13.3,1.1,0.0,13.343106,16.756864,2025-06,2025,6,summer,Wednesday
3620,2025-06-12,1,58,16.6,15.2,0.0,0.0,15.379931,16.773250,2025-06,2025,6,summer,Thursday
3621,2025-06-13,3,42,18.7,15.9,0.0,0.0,15.390986,16.787853,2025-06,2025,6,summer,Friday
3622,2025-06-14,0,42,23.0,21.0,0.0,0.0,15.412967,16.800631,2025-06,2025,6,summer,Saturday


In [15]:
# add weather code in words

with open("raw/descriptions.json") as f:
    weather_dict = json.load(f)

In [16]:
weather_dict

{'0': {'day': {'description': 'Sunny',
   'image': 'http://openweathermap.org/img/wn/01d@2x.png'},
  'night': {'description': 'Clear',
   'image': 'http://openweathermap.org/img/wn/01n@2x.png'}},
 '1': {'day': {'description': 'Mainly Sunny',
   'image': 'http://openweathermap.org/img/wn/01d@2x.png'},
  'night': {'description': 'Mainly Clear',
   'image': 'http://openweathermap.org/img/wn/01n@2x.png'}},
 '2': {'day': {'description': 'Partly Cloudy',
   'image': 'http://openweathermap.org/img/wn/02d@2x.png'},
  'night': {'description': 'Partly Cloudy',
   'image': 'http://openweathermap.org/img/wn/02n@2x.png'}},
 '3': {'day': {'description': 'Cloudy',
   'image': 'http://openweathermap.org/img/wn/03d@2x.png'},
  'night': {'description': 'Cloudy',
   'image': 'http://openweathermap.org/img/wn/03n@2x.png'}},
 '45': {'day': {'description': 'Foggy',
   'image': 'http://openweathermap.org/img/wn/50d@2x.png'},
  'night': {'description': 'Foggy',
   'image': 'http://openweathermap.org/img/wn/50

In [17]:
df['weather_code'] = df['weather_code'].astype(str)

In [18]:
df['weather_description'] = df['weather_code'].map( lambda x: weather_dict.get(x, {}).get('day', {}).get('description', 'Unknown'))
df['weather_description']

0              Cloudy
1       Light Drizzle
2              Cloudy
3       Light Drizzle
4                Rain
            ...      
3619    Light Drizzle
3620     Mainly Sunny
3621           Cloudy
3622            Sunny
3623          Drizzle
Name: weather_description, Length: 3624, dtype: object

In [19]:
# save cleaned file with calculations

df.to_csv('processed/berlin_cleaned.csv', index=False)